## Import Library

In [67]:
import pandas as pd
import re

## Load data

In [68]:
file_path = r'C:\Users\user\CAPSTONE-PROJEK---CODEY\Machine Learning\kategori soal\function.csv'

data = pd.read_csv(file_path)

data.head()

,Instruction,Input,Output,materi,question_id
0,Generate a function in Python to convert a giv...,17,def convertToBinary(n): \n \n if n > 1: \n...,Function,14
1,Create a function in Python to compute a Fibon...,10,"def fibonacci(n): \n if n < 0: \n print(""Inco...",Function,18
2,Construct a function that will find the value ...,"dictionary = {'key1':1, 'key2':2, 'key3':3}\nk...","def find_value(dictionary, key):\n value = ...",Function,21
3,Design a Python function that converts strings...,This is an example sentence,def convertToCapitalize(sentence)\n words =...,Function,25
4,Create an AWS Lambda function in Python that a...,"{\n ""x"": 10,\n ""y"": 20\n}","import json\n\ndef handler(event, context):\n ...",Function,52


In [69]:
def add_placeholder(code):
    """
    Gantikan bagian tertentu dalam kode dengan placeholder '___', kecuali jika baris mengandung 'print'.
    """
    # Pisahkan kode menjadi baris-baris
    lines = code.split('\n')
    
    # Pola regex yang akan dicari
    patterns = [
        r'def\s+\w+\s*\(.*\)\s*:',  # Deklarasi fungsi
        r'\w+\s*\(.*\)',             # Pemanggilan fungsi
        r'\(\s*(\w+\s*,\s*)*\w+\s*\)',  # Fungsi dengan parameter
        r'def\s+\w+\(\)\s*:',        # Fungsi tanpa parameter
    ]
    
    # Ganti baris yang cocok dengan pola menjadi '___', kecuali jika mengandung 'print'
    for i, line in enumerate(lines):
        # Cek apakah baris mengandung kata 'print'
        if 'print' not in line:
            for pattern in patterns:
                if re.search(pattern, line):
                    lines[i] = re.sub(pattern, '___', line)  # Ganti dengan placeholder '___'
    
    # Gabungkan kembali baris-baris menjadi satu string
    return "\n".join(lines)


Menambah kolom Template untuk soal 

In [70]:
# Terapkan fungsi pada kolom 'Full Code'
data['Template'] = data['Output'].apply(add_placeholder)
print(data['Template'][0])
print(data['Output'][0])

def convertToBinary___: 
  
    if n > 1: 
        ___ 
  
    print(n % 2,end = '') 
  

# driver code 
num = 17
convertToBinary___
def convertToBinary(n): 
  
    if n > 1: 
        convertToBinary(n//2) 
  
    print(n % 2,end = '') 
  

# driver code 
num = 17
convertToBinary(num)


Membuat fungsi untuk mengecek data soal memiliki placeholder

In [71]:
def check_placeholder_in_column(df, column_name, placeholder="___"):
    """
    Cek apakah kolom tertentu dalam DataFrame mengandung placeholder.
    
    Args:
        df (pd.DataFrame): DataFrame yang akan dicek.
        column_name (str): Nama kolom yang akan diperiksa.
        placeholder (str): Placeholder yang dicari (default: "___").
    
    Returns:
        pd.Series: Kolom boolean (True jika ada placeholder, False jika tidak).
    """
    return df[column_name].str.contains(placeholder, na=False)


In [72]:
data["has_placeholder"] = check_placeholder_in_column(data, "Template")

print(data["has_placeholder"].value_counts())

has_placeholder
True     497
False     15
Name: count, dtype: int64


In [73]:
# Hapus baris dengan nilai False
data = data[data["has_placeholder"]]

print(data["has_placeholder"].value_counts())

# Hapus kolom 'has_placeholder' jika tidak diperlukan lagi
data = data.drop(columns=["has_placeholder"])


has_placeholder
True    497
Name: count, dtype: int64


## Save Data

In [74]:
# Simpan dataset yang telah diproses
data.to_csv('Function_with_placeholders.csv', index=False)

In [75]:
data.head()

,Instruction,Input,Output,materi,question_id,Template
0,Generate a function in Python to convert a giv...,17,def convertToBinary(n): \n \n if n > 1: \n...,Function,14,def convertToBinary___: \n \n if n > 1: \n...
1,Create a function in Python to compute a Fibon...,10,"def fibonacci(n): \n if n < 0: \n print(""Inco...",Function,18,"def fibonacci___: \n if n < 0: \n print(""Inco..."
2,Construct a function that will find the value ...,"dictionary = {'key1':1, 'key2':2, 'key3':3}\nk...","def find_value(dictionary, key):\n value = ...",Function,21,def find_value___:\n value = dictionary.get...
3,Design a Python function that converts strings...,This is an example sentence,def convertToCapitalize(sentence)\n words =...,Function,25,def convertToCapitalize___\n words = senten...
4,Create an AWS Lambda function in Python that a...,"{\n ""x"": 10,\n ""y"": 20\n}","import json\n\ndef handler(event, context):\n ...",Function,52,"import json\n\ndef handler___:\n x = event[""x""..."
